In [68]:
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem
from pyzernike import ZernikeDescriptor


def create_hard_voxel_from_sdf(mol, grid_width=0.5, weight_by_mass=True):
    # 1. Загрузка молекулы

    # Добавляем водороды, если их нет, для полноты картины
    #mol = Chem.AddHs(mol)
    # Если в SDF нет 3D координат, генерируем их
    if mol.GetNumConformers() == 0:
        AllChem.EmbedMolecule(mol, AllChem.ETKDG())

    pt = Chem.GetPeriodicTable()
    conf = mol.GetConformer()
    xyz = conf.GetPositions()

    # Центрируем молекулу
    center_mass = np.mean(xyz, axis=0)
    xyz_centered = xyz - center_mass

    # 2. Определение размеров куба
    max_dist = np.max(np.linalg.norm(xyz_centered, axis=1))
    # Берем запас под радиус самого крупного атома (~2.5A)
    cube_side_angstrom = (max_dist + 3.0) * 2
    cube_size = int(np.ceil(cube_side_angstrom / grid_width))

    # Создаем пустой куб
    voxel_cube = np.zeros((cube_size, cube_size, cube_size), dtype=np.float32)
    grid_center = cube_size // 2

    # 3. Заполнение "шариков"
    for i, atom in enumerate(mol.GetAtoms()):
        symbol = atom.GetSymbol()
        # Вес: либо атомная масса, либо просто 1 (по желанию)
        weight = pt.GetAtomicWeight(symbol) if weight_by_mass else 1.0
        radius = pt.GetRvdw(symbol)

        # Индексы сетки для центра атома
        atom_grid_pos = np.round(xyz_centered[i] / grid_width).astype(int) + grid_center

        # Определяем область поиска вокруг атома в вокселях
        r_vox = int(np.ceil(radius / grid_width))

        # Генерируем локальную сетку индексов
        z, y, x = np.ogrid[-r_vox : r_vox+1, -r_vox : r_vox+1, -r_vox : r_vox+1]

        # Маска шара: x^2 + y^2 + z^2 <= r^2
        dist_sq = (x**2 + y**2 + z**2) * (grid_width**2)
        mask = dist_sq <= (radius**2)

        # Координаты для вставки в основной куб
        z_s, z_e = atom_grid_pos[0]-r_vox, atom_grid_pos[0]+r_vox+1
        y_s, y_e = atom_grid_pos[1]-r_vox, atom_grid_pos[1]+r_vox+1
        x_s, x_e = atom_grid_pos[2]-r_vox, atom_grid_pos[2]+r_vox+1

        # Добавляем массу атома в воксели, попавшие в радиус
        # Используем += для корректной обработки перекрытий
        voxel_cube[z_s:z_e, y_s:y_e, x_s:x_e][mask] += weight

    return voxel_cube


ModuleNotFoundError: No module named 'pyzernike'

In [ ]:
MOL_A = path_to_data / 'output_sdf_files' / 'C28H33N3O2.sdf'

# Загрузка молекулы через RDKit (SDF, PDB или Mol2)
mol = Chem.MolFromMolFile(MOL_A)
#mol = Chem.MolFromXYZFile(str(MOL_A))

voxel_cube = create_hard_voxel_from_sdf(mol, grid_width=0.3)


In [ ]:
import plotly.graph_objects as go
voxel = voxel_cube
# Создаем координатную сетку
x, y, z = np.meshgrid(
    np.arange(voxel.shape[0]),
    np.arange(voxel.shape[1]),
    np.arange(voxel.shape[2]),
    indexing='ij'  # важно для правильного порядка
)

# Преобразуем в 1D массивы
x_flat = x.flatten()
y_flat = y.flatten()
z_flat = z.flatten()
values_flat = voxel.flatten()
# Визуализация
fig = go.Figure(data=go.Isosurface(
    x=x_flat,
    y=y_flat,
    z=z_flat,
    value=values_flat,
    isomin=voxel.mean(),  # Попробуйте разные значения
    isomax=voxel.max() * 0.5,  # Половина от максимума
    surface_count=3,  # Количество изоповерхностей
    opacity=0.6,
    caps=dict(x_show=False, y_show=False, z_show=False),
    colorscale='viridis'
))

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1)
    ),
    title=f"3D Voxel Visualization: {voxel.shape}"
)

fig.show()


In [ ]:
order = 10
descriptor = ZernikeDescriptor.fit(data=voxel_cube, order=order)
coeffs = descriptor.get_coefficients()

print(f"Куб размера {voxel_cube.shape} успешно превращен в дескриптор из {len(coeffs)} чисел.")
print(f"Коэффициенты Цернике: {coeffs}")
